In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks':
  os.chdir(r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/3s/vv1d0lmn7g134m4psncn2_q80000gn/T/ipykernel_13447/1023205354.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap.py": 1685228246.3990028, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap_external.py": 1685228246.4039905, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codecs.py": 1685228243.6488864, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/aliases.py": 1685228245.1487875, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/__init__.py": 1685228245.143839, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/utf_8.py": 1685228245.728661, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library

In [2]:
import optuna
import pandas as pd
import numpy as np
import pickle

from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [3]:
input_path = "../data/processed/"
output_path = "../data/processed/"

training_path = input_path + "train_data/"
test_path = input_path + "test_data/"


results_path = "../models/results/"
figures_path = "../../Memoria/figures/"

In [4]:
# FILES
results_file = "results_table.csv"

In [5]:
dataset_name = "maccs_keys"

In [6]:
# LOAD DATA
train = pd.read_csv(training_path + dataset_name + "_training.csv")
test = pd.read_csv(test_path + dataset_name + "_test.csv")

X_train = train.drop(columns=["activity"])
Y_train = train["activity"]

X_test = test.drop(columns=["activity"])
Y_test = test["activity"]


In [7]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 3, 50, step=1)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.1)
    gamma = trial.suggest_float("gamma", 0.0, 2.0, step=0.1)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 20, step=1)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 2.0, step=0.1)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 2.0, step=0.1)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        n_jobs=-1,
    )

    score = cross_val_score(model, X_train, Y_train, scoring="roc_auc", cv=5, n_jobs=-1)
    auc = score.mean()

    return auc


In [8]:
# OPTUNA STUDY
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=300)

[I 2023-05-30 00:45:09,051] A new study created in memory with name: no-name-153a0006-8fbb-4e38-8c0d-cc984e416691


[I 2023-05-30 00:45:14,273] Trial 0 finished with value: 0.5276422065647018 and parameters: {'n_estimators': 100, 'max_depth': 13, 'learning_rate': 0.06999999999999999, 'subsample': 0.8, 'colsample_bytree': 0.9, 'gamma': 1.4000000000000001, 'min_child_weight': 16, 'reg_alpha': 0.1, 'reg_lambda': 0.7000000000000001}. Best is trial 0 with value: 0.5276422065647018.


[I 2023-05-30 00:45:15,595] Trial 1 finished with value: 0.5516583605430487 and parameters: {'n_estimators': 450, 'max_depth': 49, 'learning_rate': 0.01, 'subsample': 0.7, 'colsample_bytree': 0.6, 'gamma': 1.2000000000000002, 'min_child_weight': 11, 'reg_alpha': 1.9000000000000001, 'reg_lambda': 1.4000000000000001}. Best is trial 1 with value: 0.5516583605430487.


[I 2023-05-30 00:45:16,550] Trial 2 finished with value: 0.5911067193675889 and parameters: {'n_estimators': 100, 'max_depth': 19, 'learning_rate': 0.06999999999999999, 'subsample': 0.6, 'colsample_bytree': 0.5, 'gamma': 0.2, 'min_child_weight': 4, 'reg_alpha': 0.0, 'reg_lambda': 0.2}. Best is trial 2 with value: 0.5911067193675889.


[I 2023-05-30 00:45:17,972] Trial 3 finished with value: 0.5043478260869565 and parameters: {'n_estimators': 750, 'max_depth': 30, 'learning_rate': 0.03, 'subsample': 0.9, 'colsample_bytree': 0.5, 'gamma': 0.4, 'min_child_weight': 20, 'reg_alpha': 2.0, 'reg_lambda': 1.6}. Best is trial 2 with value: 0.5911067193675889.


...

[I 2023-05-30 00:52:06,116] Trial 295 finished with value: 0.6436672967863893 and parameters: {'n_estimators': 450, 'max_depth': 7, 'learning_rate': 0.04, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 1.3, 'min_child_weight': 2, 'reg_alpha': 0.5, 'reg_lambda': 0.9}. Best is trial 234 with value: 0.6672881938477402.


[I 2023-05-30 00:52:07,810] Trial 296 finished with value: 0.637523629489603 and parameters: {'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 1.2000000000000002, 'min_child_weight': 2, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 1.0}. Best is trial 234 with value: 0.6672881938477402.


[I 2023-05-30 00:52:08,658] Trial 297 finished with value: 0.6143409520536174 and parameters: {'n_estimators': 400, 'max_depth': 4, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 1.1, 'min_child_weight': 6, 'reg_alpha': 0.5, 'reg_lambda': 1.0}. Best is trial 234 with value: 0.6672881938477402.


[I 2023-05-30 00:52:10,170] Trial 298 finished with value: 0.6582230623818525 and parameters: {'n_estimators': 450, 'max_depth': 8, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 1.3, 'min_child_weight': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.9}. Best is trial 234 with value: 0.6672881938477402.


[I 2023-05-30 00:52:11,083] Trial 299 finished with value: 0.6644010998453342 and parameters: {'n_estimators': 350, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 1.2000000000000002, 'min_child_weight': 1, 'reg_alpha': 0.4, 'reg_lambda': 1.0}. Best is trial 234 with value: 0.6672881938477402.


In [9]:
# BEST OPTUNA MODEL
best_model = study.best_trial.params

In [10]:
# EVALUATE BEST MODEL ON TEST SET
model = XGBClassifier(
    n_estimators=best_model["n_estimators"],
    max_depth=best_model["max_depth"],
    learning_rate=best_model["learning_rate"],
    subsample=best_model["subsample"],
    colsample_bytree=best_model["colsample_bytree"],
    gamma=best_model["gamma"],
    min_child_weight=best_model["min_child_weight"],
    reg_alpha=best_model["reg_alpha"],
    reg_lambda=best_model["reg_lambda"],
    random_state=42,
    n_jobs=-1,
)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_proba = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(Y_test, Y_pred_proba)

print("AUC: ", auc)

AUC:  0.8380893300248138


In [11]:
# EVALUTE BEST MODEL
score = cross_val_score(model, X_train, Y_train, scoring="roc_auc", cv=10, n_jobs=-1)
auc = score.mean()

In [12]:
# SAVE MODEL FOR DEPLOYMENT
pickle.dump(model, open("../models/xgboost_model.pkl", "wb"))